In [1]:
import tweepy
import pandas as pd
import time
from sqlalchemy import create_engine
from sqlalchemy import VARCHAR, FLOAT, DATE
import numpy as np

In [2]:
#twitter keys and connecting to tweepy and the twitter API
twitter_keys={
'consumer_key':'************',
'consumer_secret':'**************',
'access_token_key':'**************',
'access_token_secret':'*******************************'
}
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'],twitter_keys['access_token_secret'])
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True,compression=True)

In [7]:
#method to get tweets and afterward parse it into database
def get_tweets(username,count):
    tweets = api.user_timeline(screen_name =username,count=count,tweet_mode='extended',exclude_replies=True)
    user = api.get_user(username)
    ID = user.id_str
    con = create_engine('mysql+pymysql://********@**********/examination_sma')
    conn=con.connect()
    for tweet in tweets:
        text_tweet = tweet.full_text.replace('"','""').replace('%','%%').replace("'","''")
        conn.execute('INSERT INTO twitter (tweet_id,user_id,username,tweet_content,favorites_count,retweets_count) VALUES ("{0}","{1}","{2}","{3}",{4},{5})'.format(str(tweet.id_str),str(ID),str(username),str(text_tweet),tweet.favorite_count,tweet.retweet_count))
    conn.close()
    con.dispose()

In [9]:
get_tweets("JoeBiden",1000)

In [10]:
get_tweets("DavidDobrik",1000)

In [11]:
get_tweets("GretaThunberg",1000)

In [79]:
#get comments of the tweets for specific user
def get_comments(username,user_id):
    replies=[]
    for tweet in tweepy.Cursor(api.search,q='to:'+username,result_type='recent',tweet_mode='extended',timeout=999999).items(1000):
        if hasattr(tweet,'in_reply_to_status_id_str'):
            replies.append(tweet.full_text)
    con = create_engine('mysql+pymysql://***********@***********+/examination_sma')
    conn=con.connect()
    for tweet in replies:
        text_tweet = tweet.replace('"','""').replace('%','%%').replace("'","''")
        text_tweet = text_tweet.split("@"+username)[1:]
        text_tweet = ''.join(text_tweet)
        conn.execute('INSERT INTO twitter_comments (user_id,username,comment) VALUES ("{0}","{1}","{2}")'.format(str(user_id),str(username),str(text_tweet)))
    conn.close()
    con.dispose()

In [80]:
#here you can see, that i struggled with rate limits
df_user = pd.read_sql_query("SELECT DISTINCT username,user_id FROM twitter",con = create_engine('mysql+pymysql://********@********/examination_sma'))
for i in range(len(df_tweets)):
    get_comments(df_user.at[i,'username'],df_user.at[i,'user_id'])

Rate limit reached. Sleeping for: 720
